In [4]:
from pyo import *

In [5]:
# constantes del sonido
CHUNK = 1024  # Número de muestras por buffer
SRATE = 44100  # Frecuencia de muestreo
AMP = 0.1 # Amplitud general del sonido
DEVICE_OUT = 6  # Dispositivo de audio a utilizar (puede cambiar según el sistema)
DEVICE_IN = 6  # Dispositivo de audio a utilizar (puede cambiar según el sistema)

In [6]:
# Funcion para iniciar el servidor de audio
def start_server():
    server = Server(nchnls=1)
    server.amp = AMP
    server.setSamplingRate(SRATE)
    server.setBufferSize(CHUNK)
    server.setInputDevice(DEVICE_IN) # & = default
    server.setOutputDevice(DEVICE_OUT) # & = default
    server.setNchnls(1)  # Número de canales de salida aparentemente solo vale 1
    server.boot()
    # server.start()
    return server

In [7]:
raise Exception("Esta es una excepción lanzada intencionalmente.")

Exception: Esta es una excepción lanzada intencionalmente.

In [ ]:
# mostar dispositivos de entrada y salida
pa_list_devices()


AUDIO devices:
0: OUT, name: HDA Intel HDMI: 0 (hw:0,3), host api index: 0, default sr: 44100 Hz, latency: 0.008707 s
1: OUT, name: HDA Intel HDMI: 1 (hw:0,7), host api index: 0, default sr: 44100 Hz, latency: 0.008707 s
2: OUT, name: HDA Intel HDMI: 2 (hw:0,8), host api index: 0, default sr: 44100 Hz, latency: 0.008707 s
3: IN, name: HDA Intel PCH: ALC3235 Analog (hw:1,0), host api index: 0, default sr: 44100 Hz, latency: 0.008707 s
4: OUT, name: hdmi, host api index: 0, default sr: 44100 Hz, latency: 0.008707 s
5: IN, name: pulse, host api index: 0, default sr: 44100 Hz, latency: 0.008707 s
5: OUT, name: pulse, host api index: 0, default sr: 44100 Hz, latency: 0.008707 s
6: IN, name: default, host api index: 0, default sr: 44100 Hz, latency: 0.008707 s
6: OUT, name: default, host api index: 0, default sr: 44100 Hz, latency: 0.008707 s



ALSA lib pcm_dsnoop.c:567:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1000:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
ALSA lib pcm

In [ ]:
# Ejemplo mega simple de una onda FM en pyo
server = start_server()
freq = Sine(10, mul=0.5)
onda = Sine(440, mul=0.1, phase=freq).out() # crear una onda de 440 Hz modulada por una onda de 10 Hz
sc = Scope(onda) # crear un osciloscopio para ver la onda
server.gui(locals())

ALSA lib pcm_dsnoop.c:567:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1000:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
ALSA lib pcm

In [8]:
from pyo import *
from random import random


class Synth:
    def __init__(self, transpo=1, mul=1):
        # Transposition factor.
        self.transpo = Sig(transpo)
        # Receive midi notes, convert pitch to Hz and manage 10 voices of polyphony.
        self.note = Notein(poly=10, scale=1, first=0, last=127)

        # Handle pitch and velocity (Notein outputs normalized amplitude (0 -> 1)).
        self.pit = self.note["pitch"] * self.transpo
        self.amp = MidiAdsr(self.note["velocity"], attack=0.001, decay=0.1, sustain=0.7, release=1, mul=0.1,)

        # Anti-aliased stereo square waves, mixed from 10 streams to 1 stream
        # to avoid channel alternation on new notes.
        self.osc1 = LFO(self.pit, sharp=0.5, type=2, mul=self.amp).mix(1)
        self.osc2 = LFO(self.pit * 0.997, sharp=0.5, type=2, mul=self.amp).mix(1)

        # Stereo mix.
        self.mix = Mix([self.osc1, self.osc2], voices=2)

        # High frequencies damping.
        self.damp = ButLP(self.mix, freq=5000)

        # Moving notches, using two out-of-phase sine wave oscillators.
        self.lfo = Sine(0.2, phase=[random(), random()]).range(250, 4000)
        self.notch = ButBR(self.damp, self.lfo, mul=mul)

    def out(self):
        "Sends the synth's signal to the audio output and return the object itself."
        self.notch.out()
        return self

    def sig(self):
        "Returns the synth's signal for future processing."
        return self.notch


s = start_server()
s.setMidiInputDevice(99)  # Open all input devices.
s.boot()

# Create the midi synth.
a1 = Synth()

# Send the synth's signal into a reverb processor.
rev = STRev(a1.sig(), inpos=[0.1, 0.9], revtime=2, cutoff=4000, bal=0.15).out()

# It's very easy to double the synth sound!
# One octave lower and directly sent to the audio output.
a2 = Synth(transpo=0.5, mul=0.7).out()

s.gui(locals())

ALSA lib pcm_dsnoop.c:567:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1000:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
Cannot connect to server socket err = No such file or directory
Cannot connect to server request channel
jack server is not running or cannot be started
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
JackShmReadWritePtr::~JackShmReadWritePtr - Init not done for -1, skipping unlock
ALSA lib pcm

Pyo error: Server already booted!


: 